In [ ]:
!git clone https://github.com/sumanthd17/image-descriptor.git

Cloning into 'image-descriptor'...
remote: Enumerating objects: 156, done.
remote: Counting objects: 100% (156/156), done.
remote: Compressing objects: 100% (92/92), done.
remote: Total 156 (delta 76), reused 136 (delta 56), pack-reused 0
Receiving objects: 100% (156/156), 158.22 KiB | 8.33 MiB/s, done.
Resolving deltas: 100% (76/76), done.


In [ ]:
rm -r train2014/

In [ ]:
!wget http://images.cocodataset.org/zips/train2014.zip
!unzip train2014

Streaming output truncated to the last 5000 lines.
 extracting: train2014/COCO_train2014_000000013714.jpg  
 extracting: train2014/COCO_train2014_000000194043.jpg  
 extracting: train2014/COCO_train2014_000000219859.jpg  
 extracting: train2014/COCO_train2014_000000278135.jpg  
 extracting: train2014/COCO_train2014_000000141015.jpg  
 extracting: train2014/COCO_train2014_000000280923.jpg  
 extracting: train2014/COCO_train2014_000000200024.jpg  
 extracting: train2014/COCO_train2014_000000435713.jpg  
 extracting: train2014/COCO_train2014_000000249993.jpg  
 extracting: train2014/COCO_train2014_000000424160.jpg  
 extracting: train2014/COCO_train2014_000000142761.jpg  
 extracting: train2014/COCO_train2014_000000532668.jpg  
 extracting: train2014/COCO_train2014_000000564904.jpg  
 extracting: train2014/COCO_train2014_000000346384.jpg  
 extracting: train2014/COCO_train2014_000000560934.jpg  
 extracting: train2014/COCO_train2014_000000122356.jpg  
 extracting: train2014/COCO_train2014

In [ ]:
!wget http://images.cocodataset.org/annotations/annotations_trainval2014.zip
!unzip annotations_trainval2014

--2020-07-06 09:50:20--  http://images.cocodataset.org/annotations/annotations_trainval2014.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 52.216.102.43
Connecting to images.cocodataset.org (images.cocodataset.org)|52.216.102.43|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 252872794 (241M) [application/zip]
Saving to: ‘annotations_trainval2014.zip’

annotations_trainva 100%[===================>] 241.16M  47.0MB/s    in 5.6s    

2020-07-06 09:50:26 (43.4 MB/s) - ‘annotations_trainval2014.zip’ saved [252872794/252872794]

Archive:  annotations_trainval2014.zip
  inflating: annotations/instances_train2014.json  
  inflating: annotations/instances_val2014.json  
  inflating: annotations/person_keypoints_train2014.json  
  inflating: annotations/person_keypoints_val2014.json  
  inflating: annotations/captions_train2014.json  
  inflating: annotations/captions_val2014.json  


In [ ]:
cd image-descriptor

/content/image-descriptor


In [ ]:
from utils.vocabulary import Vocabulary

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
v = Vocabulary(5, vocab_file='vocab.pkl', from_vocab_file=False, annotation_file='data/annotations/captions_train2014.json')

loading annotations into memory...
Done (t=0.59s)
creating index...


814it [00:00, 8135.93it/s]

index created!


414113it [00:43, 9608.33it/s]


In [ ]:
!pwd

/content/image-descriptor


In [ ]:
import math
import time
import sys
import os

import torch
import torch.nn as nn
import torch.utils.data as data

import numpy as np

from utils.transforms import transform_train, transform_val
from utils.dataloader import dataloader

from models.encoders import ResNet50
from models.decoders import TextualHead

In [ ]:
PRINT_EVERY = 1000

In [ ]:
def train(
    train_loader,
    encoder,
    decoder,
    criterion,
    optimizer,
    vocab_size,
    epoch,
    total_step,
    start_step=1,
    start_loss=0.0,
):
    encoder.train()
    decoder.train()

    total_loss = start_loss

    start_train_time = time.time()

    for i_step in range(start_step, total_step + 1):
        indices = train_loader.dataset.get_indices()
        new_sampler = data.sampler.SubsetRandomSampler(indices=indices)
        train_loader.batch_sampler.sampler = new_sampler

        for batch in train_loader:
            images, captions = batch[0], batch[1]
            break
        mask = captions.shape[1] * torch.ones(32)
        if torch.cuda.is_available():
            images = images.cuda()
            captions = captions.cuda()
            mask = mask.cuda()

        features = encoder(images)
        outputs = decoder(features, captions, mask)

        loss = criterion(outputs.view(-1, vocab_size), captions.view(-1))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        stats = "Epoch %d, Train step [%d/%d], %ds, Loss: %.4f, Perplexity: %5.4f" % (
            epoch,
            i_step,
            total_step,
            time.time() - start_train_time,
            loss.item(),
            np.exp(loss.item()),
        )
        print("\r" + stats, end="")
        sys.stdout.flush()

        if i_step % PRINT_EVERY == 0:
            print("\r" + stats)
            filename = os.path.join(
                "./models", "train-model-{}{}.pkl".format(epoch, i_step)
            )
            torch.save(
                {
                    "encoder": encoder.state_dict(),
                    "decoder": decoder.state_dict(),
                    "optimizer": optimizer.state_dict(),
                    "total_loss": total_loss,
                    "epoch": epoch,
                    "train_step": i_step,
                },
                filename,
            )
            start_train_time = time.time()

    return total_loss / total_step

In [ ]:
dl = dataloader(
    mode="train",
    transform=transform_val,
    batch_size=32,
    vocab_threshold=5,
    vocab_file="./pickle/vocab.pkl",
    from_vocab_file=True,
    img_dir_path="./data/train2014",
    captions_path="./data/annotations/captions_train2014.json",
)

loading annotations into memory...
Done (t=0.60s)
creating index...


  0%|          | 861/414113 [00:00<00:48, 8607.05it/s]

index created!


100%|██████████| 414113/414113 [00:41<00:00, 10084.30it/s]


In [ ]:
len(dl.dataset.vocab)

8099

In [ ]:
visual = ResNet50(1024)
textual = TextualHead(len(dl.dataset.vocab), 1024, 1, 16, 4096, 0.1, 50, 0)

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/checkpoints/resnet50-19c8e357.pth


In [ ]:
criterion = nn.CrossEntropyLoss()

params = list(visual.parameters()) + list(textual.parameters())
optimizer = torch.optim.Adam(params=params, lr=0.01)

In [ ]:
if torch.cuda.is_available():
    visual.cuda()
    textual.cuda()
    criterion.cuda()

In [ ]:
total_val_step = math.ceil(len(dl.dataset.caption_lengths) / 32)

In [ ]:
train(
    dl, visual, textual, criterion, optimizer, len(dl.dataset.vocab), 1, total_val_step
)

Epoch 1, Train step [1000/12942], 625s, Loss: 5.2509, Perplexity: 190.7299
Epoch 1, Train step [2000/12942], 615s, Loss: 4.9611, Perplexity: 142.7576
Epoch 1, Train step [3000/12942], 610s, Loss: 4.7975, Perplexity: 121.2103
Epoch 1, Train step [4000/12942], 610s, Loss: 4.6529, Perplexity: 104.8837
Epoch 1, Train step [5000/12942], 610s, Loss: 4.8853, Perplexity: 132.3317
Epoch 1, Train step [6000/12942], 612s, Loss: 4.6377, Perplexity: 103.3016
Epoch 1, Train step [7000/12942], 611s, Loss: 4.9238, Perplexity: 137.5264
Epoch 1, Train step [8000/12942], 611s, Loss: 4.7521, Perplexity: 115.8261
Epoch 1, Train step [9000/12942], 611s, Loss: 4.7331, Perplexity: 113.6482
Epoch 1, Train step [10000/12942], 612s, Loss: 4.6852, Perplexity: 108.3372
Epoch 1, Train step [11000/12942], 612s, Loss: 4.8427, Perplexity: 126.8156
Epoch 1, Train step [12000/12942], 614s, Loss: 4.9792, Perplexity: 145.3606
Epoch 1, Train step [12942/12942], 578s, Loss: 4.8728, Perplexity: 130.6869

4.890840867261691

In [ ]:
dl